# Preparing Hamiltonians for Quantum Chemistry

In this notebook, I construct an electronic structure Hamiltonian suitable for execution on quantum hardware. The workflow is implemented directly using PySCF, NumPy, and Qiskit in order to make each transformation from a molecular description to a qubit operator explicit.

Starting from a defined molecular system (geometry, spin, basis set, and active space), I build the corresponding fermionic Hamiltonian using creation and annihilation operators and then map it to a qubit Hamiltonian expressed as a weighted sum of Pauli strings. This fermion-to-qubit mapping is required because electrons obey fermionic statistics, while quantum hardware operates on qubits with different algebraic properties.

The resulting qubit Hamiltonian serves as the input for variational algorithms such as the Variational Quantum Eigensolver (VQE), which are explored in subsequent notebooks.


# My End Goal

I would like to obtain something of the form;

In [2]:
H = [(1, "XX"), (1, "YY"), (1, "ZZ")]
print(H)

#or 

from qiskit.quantum_info import SparsePauliOp
 
H = SparsePauliOp(["XX", "YY", "ZZ"], coeffs=[1.0 + 0.0j, 1.0 + 0.0j, 1.0 + 0.0j])
print(H)

[(1, 'XX'), (1, 'YY'), (1, 'ZZ')]
SparsePauliOp(['XX', 'YY', 'ZZ'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j])


### Generating Quantum Chemistry Hamiltonian On IBM Platform Requires;
    - Define your molecule (Geometry, Spin, Active Space etc)
    - Generate fermionic hamiltonians (creation and annihilation operators)
    - Map fermionic Hamiltonians to qubits using pauli operators
    - If using a thirdparty software handle syntax

PySCF (Python-Based Simulation of Chemistry Framework) ; Has a wide collection of electronic structure modules which can be used to generate molecular hamiltoninan for quantum simulation 

* Specify atomic cartesian coordinates
* gto; generate gaussian type orbitals
* basis; the function used to model molecular orbitals with sto-6g (Slater Type Orbital using 6 primitive guassian orbitals
* Spin; the integer indicating the number of unpaired electrons (2S) some software use multiplicity (2S + 1)
* Charge; the charge of the molecule
* Symmetry; specified with a string or automatically identified using symmetry = True

# 1. Define The Molecule

In [12]:
import numpy as np
from pyscf import ao2mo, gto, mcscf, scf
from qiskit.quantum_info import SparsePauliOp

In [15]:
distance = 0.735
a = distance / 2
mol = gto.Mole()
mol.build(
    verbose=4,
    atom=[
        ["H", (0, 0, -a)],
        ["H", (0, 0, a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry= True,
)

System: uname_result(system='Linux', node='CAS-6VK2KB4', release='6.6.87.1-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Mon Apr 21 17:08:54 UTC 2025', machine='x86_64')  Threads 22
Python 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:27:50) [GCC 13.3.0]
numpy 2.3.3  scipy 1.16.2  h5py 3.15.1
Date: Tue Dec 23 13:59:30 2025
PySCF version 2.11.0
PySCF path  /home/nigby/miniconda3/envs/QISKIT/lib/python3.13/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry True subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000  -0.367500000000 AA    0.000000000000   0.000000000000  -0.694474350778 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000   0.3675

In [2]:
distance = 0.735
a = distance / 2
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["H", (0, 0, -a)],
        ["H", (0, 0, a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Dooh",
)

In [3]:
# solving the approximate schrodinger equation of my specified molecule on a Hartree-Fock level

mf = scf.RHF(mol)      #creates a hartree fock solver 
mf.scf()               #runs self consistent field framework

print(
    mf.energy_nuc(), #this first output value i needed when explicity writting out hamiltonian becaused it is the nuclear repulsion
    mf.energy_elec()[0],
    mf.energy_tot(),
    mf.energy_tot() - mol.energy_nuc(),
)

np.float64(-1.125628668427439)

In [4]:
active_space = range(mol.nelectron // 2 - 1, mol.nelectron // 2 + 1)

# 2. Generate Fermionic Hamiltonian (Creation and Annihilation operators)
A Mathematical representation of our defined molecule

* Electrons are Fermions and so can be naturally expressed using creation and annihilation operators
* These operators enforces pauli exclusion principle; no 2e can occupy same quantum state in a quantum system


    - SCF; wide range of self consistent field methods
    - MCSCF; Multiconfigurational Self Consistent Field Package
    - ao2mo; Transformation of Atomic Orbital 2 Molecular Orbital
    - ncas; No. of Orbital in Complete Active Space
    - nelecas; no. of electrons in complete active space

In [5]:
#This code computes the Hartree–Fock energy, then performs a correlated active-space calculation (CASCI) on two orbitals to obtain a more accurate energy for the molecule.


E1 = mf.kernel()  #reruns hatree-fock
mx = mcscf.CASCI(mf, ncas=2, nelecas=(1, 1))
mo = mx.sort_mo(active_space, base=0)
E2 = mx.kernel(mo)[:2]

### Hamiltonian is often seperated into
    - ecore (electronic core)
    - h1e (single electron operators)
    - h2e (two electron energies

In [6]:
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)



In [7]:
h1e

NPArrayWithTag([[-1.26062688e+00,  2.21888282e-17],
                [ 4.34057091e-18, -4.76151148e-01]])

# 3. Mapping Fermionic Hamiltonian to IBM qubits using Pauli operators

There are multiple approaches for mapping a molecular Hamiltonian to a form suitable for quantum computation. In this notebook, I implement the **Jordan–Wigner mapping** explicitly using only PySCF, NumPy, and Qiskit. This makes the mapping process transparent and independent of higher-level abstractions, while remaining compatible with other software solutions that differ mainly in syntax.

To efficiently construct the two-electron contribution, I use a Cholesky decomposition to obtain a low-rank representation of the interaction terms, which simplifies the assembly of the qubit Hamiltonian.


In [8]:
# Cholesky Function; Low rank decomposition of the 2 electron part of our molecule's hamiltonian

def cholesky(V, eps):
    # see https://arxiv.org/pdf/1711.02242.pdf section B2
    # see https://arxiv.org/abs/1808.02625
    # see https://arxiv.org/abs/2104.08957
    no = V.shape[0]
    chmax, ng = 20 * no, 0
    W = V.reshape(no**2, no**2)
    L = np.zeros((no**2, chmax))
    Dmax = np.diagonal(W).copy()
    nu_max = np.argmax(Dmax)
    vmax = Dmax[nu_max]
    while vmax > eps:
        L[:, ng] = W[:, nu_max]
        if ng > 0:
            L[:, ng] -= np.dot(L[:, 0:ng], (L.T)[0:ng, nu_max])
        L[:, ng] /= np.sqrt(vmax)
        Dmax[: no**2] -= L[: no**2, ng] ** 2
        ng += 1
        nu_max = np.argmax(Dmax)
        vmax = Dmax[nu_max]
    L = L[:, :ng].reshape((no, no, ng))
    print(
        "accuracy of Cholesky decomposition ",
        np.abs(np.einsum("prg,qsg->prqs", L, L) - V).max(),
    )
    return L, ng

In [9]:
def identity(n):
    return SparsePauliOp.from_list([("I" * n, 1)])


#Next a function that converts fermionic operators(creation, annihilation) into qubit operators 
def creators_destructors(n, mapping="jordan_wigner"):
    c_list = []
    if mapping == "jordan_wigner":
        for p in range(n):
            if p == 0:
                ell, r = "I" * (n - 1), ""
            elif p == n - 1:
                ell, r = "", "Z" * (n - 1)
            else:
                ell, r = "I" * (n - p - 1), "Z" * p
            cp = SparsePauliOp.from_list([(ell + "X" + r, 0.5), (ell + "Y" + r, -0.5j)])
            c_list.append(cp)
    else:
        raise ValueError("Unsupported mapping.")
    d_list = [cp.adjoint() for cp in c_list]
    return c_list, d_list

### Using these 3 functions we would now create a Hamiltonian suitable for quantum computers
    - To do this we use the Build_Hamiltonian Function

In [13]:
def build_hamiltonian(ecore: float, h1e: np.ndarray, h2e: np.ndarray) -> SparsePauliOp:
    ncas, _ = h1e.shape
 
    C, D = creators_destructors(2 * ncas, mapping="jordan_wigner")
    Exc = []
    for p in range(ncas):
        Excp = [C[p] @ D[p] + C[ncas + p] @ D[ncas + p]]
        for r in range(p + 1, ncas):
            Excp.append(
                C[p] @ D[r]
                + C[ncas + p] @ D[ncas + r]
                + C[r] @ D[p]
                + C[ncas + r] @ D[ncas + p]
            )
        Exc.append(Excp)
 
    # low-rank decomposition of the Hamiltonian
    Lop, ng = cholesky(h2e, 1e-6)
    t1e = h1e - 0.5 * np.einsum("pxxr->pr", h2e)
 
    H = ecore * identity(2 * ncas)
    # one-body term
    for p in range(ncas):
        for r in range(p, ncas):
            H += t1e[p, r] * Exc[p][r - p]
    # two-body term
    for g in range(ng):
        Lg = 0 * identity(2 * ncas)
        for p in range(ncas):
            for r in range(p, ncas):
                Lg += Lop[p, r, g] * Exc[p][r - p]
        H += 0.5 * Lg @ Lg
 
    return H.chop().simplify()

In [14]:
H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  2.220446049250313e-16
SparsePauliOp(['IIII', 'IIIZ', 'IZII', 'IIZI', 'ZIII', 'IZIZ', 'IIZZ', 'ZIIZ', 'IZZI', 'ZZII', 'ZIZI', 'YYYY', 'XXYY', 'YYXX', 'XXXX'],
              coeffs=[-0.09820182+0.j,  0.1740751 +0.j,  0.1740751 +0.j, -0.2242933 +0.j,
 -0.2242933 +0.j,  0.16891402+0.j,  0.1210099 +0.j,  0.16631441+0.j,
  0.16631441+0.j,  0.1210099 +0.j,  0.17504456+0.j,  0.04530451+0.j,
  0.04530451+0.j,  0.04530451+0.j,  0.04530451+0.j])


### Things To Help Understand A Bit Better

    - We want a very small value of accuracy of cholesky decomposition (e.g ; 2.220446049250313e-16) This values tells us that the low rank form (cholesky) reproduces the hamiltonian almost exactly
    - The coefficeint tells us how strong each pauli operator contributes to the energy

Unsure of Active Space Symmetry

#### My Understanding so far
     - The choice of orbitals to include in the active space is decided by me but guided by chemistry and computational limits 
     - Notably we want to include HOMO and LUMO orbitals 
     - This active space orbital is specified using cas_space_symmetry = 